# Лабораторная работа №4
## Выполнил студент группы БВТ1905 Баев Дмитрий

Реализовать следующие структуры данных:
* Стек (stack):
    * операции для стека: 
        * инициализация;
        * проверка на пустоту;
        * добавление нового элемента в начало;
        * извлечение элемента из начала;
* Дек (двусторонняя очередь, deque):
    * операции для дека: 
        * инициализация;
        * проверка на пустоту;
        * добавление нового элемента в начало; 
        * добавление нового элемента в конец;
        * извлечение элемента из начала; 
        * извлечение элемента из конца;

Разработать программу обработки данных, содержащихся в заранее
подготовленном txt-файле, в соответствии с заданиями, применив
указанную в задании структуру данных. Результат работы программы
вывести на экран и сохранить в отдельном txt-файле.


In [1]:
# Библиотека для копирования листов, используется в Деке
import copy

In [2]:
class Stack:
    # Инициализатор
    def __init__(self, size):
        # Создаем массив указаного размера
        self.arr = ['empty'] * size
        # Кол-во занятых переменных
        self.count = 0
        
    # Добавление нового элемента в начало
    def push(self, elem):
        # Если проверка на переполнения не пройдена, возникает ошибка
        if self.count + 1 > len(self.arr): 
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        self.arr[self.count] = elem
        self.count += 1
        
    # Извлечение элемента с начала
    def pop(self):
        if self.count - 1 >= 0:
            elem = self.arr[self.count-1]
            self.arr[self.count-1] = 'empty'
            self.count -= 1
            return elem
            
    # Проверка на пустоту        
    def isEmpty(self):
        if self.count == 0:
                return True        
        return False
    
    # Вывод содержимого стека
    def print(self):
        print("Stack =", self.arr,"len =", self.count)

In [3]:

class Deque(Stack):
    # Инициализатор
    def __init__(self, size):
        self.j = -1
        Stack.__init__(self, size)
    
    # Сдвигаем все элементы массива в нужную сторону
    def _move(self, direct):
        if direct == 'right' and self.arr[len(self.arr) - 1] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = 0
            for i in range(len(self.arr)):
                if i >= len(self.arr) - self.count:
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
        if direct == 'left' and self.arr[0] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = len(self.arr) - self.count
            for i in range(len(self.arr)):
                if j < len(self.arr):
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
            
    # Добавление нового элемента в начало
    def push(self, elem):
        Deque._move(self, 'left')
        Stack.push(self, elem)
        
    # Извлечение элемента с начала    
    def pop(self):
        Deque._move(self, 'left')
        return Stack.pop(self)
    
    # Добавление нового элемента в конец
    def pushEnd(self, elem):
        if len(self.arr) < 2:
            Stack.push(self, elem)
        if self.count + 1 > len(self.arr):
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        else:
            Deque._move(self, 'right')
            self.arr[len(self.arr) - self.count - 1] = elem
            self.count += 1
            
    # Извлечение элемента с конца
    def popEnd(self):
        if len(self.arr) - (self.count - 1) <= len(self.arr):
            Deque._move(self, 'right')
            elem = self.arr[len(self.arr) - self.count] 
            self.arr[len(self.arr) - self.count] = 'empty'
            self.count -= 1
            return elem

### Задание 1 
Отсортировать строки файла, содержащие названия книг, в алфавитном порядке с
использованием 2-ух `Deque`.

In [58]:
books = []

# Функция для удаление символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

# Получаем список книг и записываем его в лист Books 
f = open('input.txt', encoding="utf8")
for line in f:
    books.append(nlDel(line))
f.close()
    
# Создаем две деки, с размером равным кол-ву книг
d1 = Deque(len(books))
d2 = Deque(len(books))

# Заполняем первую Деку книгами из списка
for book in range(len(books)):
    d1.push(books[book])

def sort(fr, to): 
    # Перекидываем первый элемент из одной деки в другую
    book = fr.pop()
    lastBook = book
    to.push(book)
    
    # Сортируем оставшие элементы, те у которых ,буквы на позиции j весит больше,
    # чем у предыдущего названия, идут в конец, иначе в начало
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = fr.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) < ord(book[j]):
                    to.pushEnd(book)
                if ord(lastBook[j]) > ord(book[j]):
                    to.push(book)
                lastBook = book
                break
            else:
                fr.push(book)
                
    # Проверяем, что все элементы отсортированы
    isSorted = True
    book = to.pop()
    lastBook = book
    fr.push(book)
    
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = to.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) > ord(book[j]):
                    isSorted = False
                fr.pushEnd(book)
                lastBook = book
                break
            else:
                to.push(book)
        
    if (not isSorted):
        return sort(fr,to)
    
# Сортируем список книг
sort(d1,d2)

# Открываем файл для записи 
f = open('output.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(d1.count):
    print(d1.pop(), file=f)
f.close()

### Задание 2
`Deque` содержит последовательность символов для шифровки сообщений. Дан
текстовый файл, содержащий зашифрованное сообщение. Пользуясь `Deque` нужном,
расшифровать текст. **Известно, что при шифровке каждый символ сообщения
заменялся следующим за ним в `Deque` по часовой стрелке через один.**


In [61]:
# Библиотека, которую использую для создания псевдослучайных комбинаций
import random

In [82]:
alphabet = list('АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя-.?!)(,:\\\'"')
random.shuffle(alphabet)
alphabet = ''.join(alphabet)

d = Deque(len(alphabet))
[d.push(alphabet[i]) for i in range(len(alphabet))]

s = 'Привет, как твои дела?'

def encode(t,d):
    t = list(t)
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            if c == t[i]:
                d.pushEnd(c)
                d.pushEnd(d.pop())
                c = d.pop()
                t[i] = c
            d.pushEnd(c)
    return ''.join(t)
    
def decode(t,d):
    t = list(t)
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            if c == t[i]:
                d.push(c)
                d.push(d.popEnd())
                c = d.popEnd()
                t[i] = c
            d.push(c)
    return ''.join(t)

print(encode(s,d), "- Зашифровонная фраза",)
print(decode(s,d), "- Расшифровка")

Фрф,Вэд ОЩО э,шф йВьЩН - Зашифровонная фраза
Привет, как твои дела? - Расшифровка


### Задание 8
Дан текстовый файл. Используя `Stack`, сформировать новый текстовый файл,
содержащий строки исходного файла, записанные в обратном порядке: первая
строка становится последней, вторая – предпоследней и т.д.

In [83]:
# Функция для удаление символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

books = []
    
# Открываем файл для записи
f = open('output.txt', encoding="utf8")
# Копируем книги из файла в список "Books"
for line in f:
    books.append(nlDel(line))
f.close()

s = Stack(len(books))

# Заполняем Stack
for i in range(len(books)):
    s.push(books[i])

# Открываем файл для записи 
f = open('output-reverse.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(s.count):
    print(s.pop(), file=f)
f.close()